# Code zu Lanczos-Verfahren mithilfe von QR-Verfahren

In [1]:
import numpy as np
from scipy.stats import unitary_group
from scipy.linalg import block_diag
from scipy.linalg import qr
import time
import matplotlib.pyplot as plt

### Funktionen zum generiern von Matrizen

In [2]:
def gen_hess(n, compl = False):
    if compl:
        a = np.random.rand(n) + np.random.rand(n)*1j
        b = np.random.rand(n-1) + np.random.rand(n-1)*1j
        c = np.random.rand(n-1) + np.random.rand(n-1)*1j
        return np.diag(b, -1) + np.diag(a,0) + np.diag(c, 1)
    else:
        A = np.zeros((n,n))
        a = np.random.rand(n)*50
        A = A + np.diag(a)
        for j in range(n-1):
            b = np.random.rand(n-j-1)*50
            A = A + np.diag(b,j+1)
        return np.diag(np.random.rand(n-1)*50, -1) + A
    

def gen_upper_tri(eig, compl = False):
    n = eig.shape[0]
    if compl:
        return np.triu(np.random.rand(n,n) + np.random.rand(n,n) * 1j, 1) + np.diag(eig)
    else:
        return np.triu(np.random.rand(n,n),1) + np.diag(eig)
    

def gen_hermite(eig, compl = False):
    n = eig.shape[0]
    B = unitary_group.rvs(n)
    return B.T.conj()@np.diag(eig)@B


def gen_rand_mat(eig, compl = False):
    n = eig.shape[0]
    if compl:
        B = np.random.rand(n,n) + np.random.rand(n,n)*compl*1j
    else:
        B = np.random.rand(n,n)
    return np.linalg.inv(B)@np.diag(eig)@B



#print(gen_hess(5))
#print(gen_upper_tri(np.array([1,2,3]), True))
#print(gen_hermite(np.array([1,2,3,4])))
#print(gen_rand_mat(np.array([1,2,7,4])))



### Verfahren implementiert

In [3]:
def QR_simple(A,tol = 1e-6):
    count = 0
    while abs(A[1,0]) > tol:
        Q,R = np.linalg.qr(A)
        A = R@Q
        count +=1
    return A, sorted(np.diag(A)) ,count



def QR_shift(A,tol=1e-10):
    n = A.shape[1]
    count = 0
    for i in range(n-1,0,-1):
        while abs(A[i,i-1]) > tol:
            rho = A[i,i]
            Q,R = np.linalg.qr(A-rho*np.identity(n))
            A = R@Q + rho*np.identity(n)
            count +=1
        A[i,:i-n] = 0
    return A, sorted(np.diag(A)), count



def QR_shift2(A,tol=1e-12):
    n = A.shape[1]
    count = 0
    for i in range(n-1,0,-1):
        while abs(A[i,i-1]) > tol*(abs(A[i-1,i-1])+abs(A[i,i])):
            w = np.linalg.eigvals(A[i-1:i+1,i-1:i+1])
            if abs(w[0] - A[i,i]) < abs(w[1] - A[i,i]):
                rho = w[0]
            else:
                rho = w[1]
            Q,R = np.linalg.qr(A-rho*np.identity(n))
            A = R@Q + rho*np.identity(n)
            count += 1
        A[i,:i-n] = 0
    return A, sorted(np.diag(A)), count



def QR_hesse(A,tol=1e-12):
    n = A.shape[1]
    count = 0
    for i in range(n-1,0,-1):
        while abs(A[i,i-1]) > tol*(abs(A[i-1,i-1])+abs(A[i,i])):
            w = np.linalg.eigvals(A[i-1:i+1,i-1:i+1])
            if abs(w[0] - A[i,i]) < abs(w[1] - A[i,i]):
                rho = w[0]
            else:
                rho = w[1]
            Q,R = QR_decomp_hesse(A-rho*np.identity(n))
            A = R@Q + rho*np.identity(n)
            count += 1
        A[i,:i-n] = 0
    return A, sorted(np.diag(A)), count



def QR_decomp_hesse(A):
    n = A.shape[0]
    Q = np.eye(n,n)
    
    for i in range(0,n-1):
        if abs(A[i,i]) >= abs(A[i+1,i]):
            t = A[i+1,i]/abs(A[i,i])
            root = (1+abs(t)**2)**(1/2)
            c = A[i,i]/(abs(A[i,i])*root)
            s = t/root
        else:
            t = A[i,i]/abs(A[i+1,i])
            root = (1+abs(t)**2)**(1/2)
            s = A[i+1,i]/(abs(A[i+1,i])*root)
            c = t/root
      
        for j in range(n):
            if j < i:
                temp_2 = Q[i,j]
                Q[i,j] = c.conj()*temp_2 + s.conj()*Q[i+1,j]
                Q[i+1,j] = -s*temp_2 + c*Q[i+1,j]
            else:
                temp_1 = A[i,j]
                A[i,j] = c.conj()*temp_1 + s.conj()*A[i+1,j]
                A[i+1,j] = -s*temp_1 + c*A[i+1,j]
                temp_2 = Q[i,j]
                Q[i,j] = c.conj()*temp_2 + s.conj()*Q[i+1,j]
                Q[i+1,j] = -s*temp_2 + c*Q[i+1,j]
    
    return Q.T.conj(), A

def QR_decomp_hesse_old(A):
    n = A.shape[0]
    Q = np.eye(n,n)

    for i in range(0,n-1):
        if abs(A[i,i]) >= abs(A[i+1,i]):
            t = A[i+1,i]/abs(A[i,i])
            root = (1+abs(t)**2)**(1/2)
            c = A[i,i]/(abs(A[i,i])*root)
            s = t/root
        else:
            t = A[i,i]/abs(A[i+1,i])
            root = (1+abs(t)**2)**(1/2)
            s = A[i+1,i]/(abs(A[i+1,i])*root)
            c = t/root
            
        M = np.array([[c.conj(), s.conj()], [-s, c]])
        G = block_diag(np.eye(i,i), M, np.eye(n-i-2, n-i-2))
        Q = G@Q
        for j in range(i, n):
            temp_1 = A[i,j]
            A[i,j] = c.conj()*temp_1 + s.conj()*A[i+1,j]
            A[i+1,j] = -s*temp_1 + c*A[i+1,j]
    return Q.T.conj(), A

def lanczos(A,k = 0):
    n = A.shape[1]
    if k == 0:
        k = n
        
    v0 = np.random.rand(n)
    v = [v0/np.linalg.norm(v0)]
    gam = [v[0].T.conj()@A@v[0]]
    w = (A - gam[0]*np.identity(n))@v[0]
    delta = [np.linalg.norm(w)]
    i = 0
    while delta[i] > 1e-10 and i < k-1:
        v.append(w/delta[i])
        i +=1
        gam.append(v[i].T.conj()@A@v[i])
        w = (A - gam[i]*np.identity(n))@v[i] - delta[i-1]*v[i-1]
        delta.append(np.linalg.norm(w))
    T = np.diag(delta[:-1], -1) + np.diag(gam) + np.diag(delta[:-1], 1)
    return QR_hesse(T)

def arnoldi(A, dim, k = 0):
    n = dim
    if k == 0:
        k = n
        
    v0 = np.random.rand(n)
    v = [v0/np.linalg.norm(v0)]
    h = np.zeros((k,k))
    
    for j in range(k):
        w = A(v[j])
        for l in range(j+1):
            h[l][j] = w.T.conj()@v[l]
            w = w - h[l][j]*v[l]
        if j < k-1:
            h[j+1][j] = np.sqrt(w.T.conj()@w)
            if abs(h[j+1][j]) < 1e-2:
                return QR_hesse(h[:j+1,:j+1])
            else:
                #print("j=",j)
                #for s in range(j):
                #    print(v[j].T.conj()@v[s])
                #print("------------------------------------")
                v.append(w/h[j+1][j])
    return QR_hesse(h)

### Testing

In [4]:
def error(eig_1, eig_2):
    eig_1 = np.sort(eig_1)
    eig_2 = np.sort(eig_2)
    return np.linalg.norm(eig_1 - eig_2)


def runtime(f,A):
    start = time.time()
    f(A)
    end = time.time()
    return end - start

def runtime_error(f,A):
    start = time.time()
    S = f(A)
    end = time.time()
    return end - start, S

def runtime_error_lanczos(f,A,k):
    start = time.time()
    S = f(A,k)
    end = time.time()
    return end - start, S

def testing(f, n_1, n_2):
    n_array = np.array([i for i in range(n_1,n_2)])
    time_array = []
    error_array = []
    for n in n_array:
        eigv = np.sort(50*np.random.rand(n)-25)
        A = gen_hermite(eigv)
        t, A = runtime_error(f,A)
        err = error(A[1], eigv)
        time_array.append(t)
        error_array.append(err)
    plt.plot(n_array, time_array, label = "runtime")
    #plt.plot(n_array, error_array, 'x', label = "errors")
    plt.legend()
    plt.show()

    
def testing_lanczos(f,k, n_1, n_2):
    n_array = np.array([i for i in range(n_1,n_2)])
    time_array = []
    error_array = []
    for n in n_array:
        eigv = np.sort(50*np.random.rand(n)-25)
        A = gen_hermite(eigv)
        t, A = runtime_error_lanczos(f,A,k)
        err = error(A[1][0], eigv[0])
        time_array.append(t)
        error_array.append(err)
    plt.plot(n_array, time_array, label = "runtime lanczos")
    #plt.plot(n_array, error_array, 'x', label = "errors")
    plt.legend()
    plt.show()
    
def selective_error(eig_1, eig_2, k = 1):
    n_1 = len(eig_1)
    n_2 = len(eig_2)
    error_1 = np.linalg.norm(eig_1[:k] - eig_2[:k])
    error_2 = np.linalg.norm(eig_1[:n_1-k-1:-1] - eig_2[:n_2-k-1:-1])
    return error_1 + error_2

# ------- testing starts here -------

n = 1000

mu, sigma = 3, 1
eigv = np.sort(np.random.lognormal(mu, sigma, n))

#eigv = np.array([-(i+1)*10000 for i in range(n)])
A = gen_hermite(eigv)

def matmul(v):
    return A@v


print(np.sort(eigv))
#print(np.round(QR_simple(A)[1],3))
#print(np.round(QR_shift(A)[1],3))
#print(np.round(QR_shift2(A)[1],3))
#print(np.round(QR_hesse(A)[0],3))
#print(np.round(lanczos(A,30)[1],3))
#print(np.linalg.eigvals(A))
#print(np.sort(np.round(arnoldi(matmul,n)[1], 3)))

#print(error(QR_simple(A)[1], eigv))
#print(error(QR_shift(A)[1], eigv))
#print(error(QR_shift2(A)[1], eigv))
#print(error(lanczos(A)[1], eigv))

#print(error(arnoldi(lambda x: A@x,n, 50)[1],eigv))
#print(arnoldi(lambda x: A@x,n, 50)[1])
#print(runtime(QR_simple, A))
#print(runtime(QR_shift, A))
#print(runtime(QR_shift2, A))
#print(runtime_error_lanczos(lanczos, A, 10)[0])


[  0.89965534   1.54078736   1.74532732   1.81123728   1.82279753
   1.86394297   1.88528301   1.99222579   2.05821364   2.06662747
   2.08006002   2.08961419   2.1165315    2.18546409   2.23819232
   2.24973914   2.33711095   2.45117262   2.49975019   2.52463784
   2.53792494   2.65334869   2.68139276   2.68841095   2.70076208
   2.74035842   2.74423432   2.79924979   2.83833867   3.07039795
   3.11351016   3.31457141   3.39132407   3.53022891   3.54709238
   3.62996593   3.72616539   3.72640174   3.74498188   3.75614314
   3.77167916   3.86146099   3.87527247   3.87869653   3.90074773
   3.95384924   4.16163036   4.30701516   4.33227753   4.36031837
   4.51528711   4.52477123   4.55305586   4.61305605   4.63376925
   4.63438437   4.64383626   4.64670693   4.6951406    4.71672499
   4.71983877   4.73635614   4.74216211   4.75703573   4.93140763
   4.9621189    4.97392802   4.99559133   5.05292154   5.11265371
   5.13385209   5.16509487   5.2198988    5.30975249   5.35083933
   5.36249

In [8]:
k = 120
for k in range(5,25):
    lanczos_test = lanczos(A,k)[1]
    print(np.sort(lanczos_test))
    err_ = error(lanczos_test[:len(lanczos_test)-4:-1], eigv[:len(eigv)-4:-1])


    print("lanczos = ", err_)

   # arnoldi_test = arnoldi(lambda x: A@x,n,k)[1]
    #print(np.sort(arnoldi_test))
   # error_1 = selective_error(arnoldi_test, eigv, 2)

    #print("arnoldi=", error_1)

C:\Users\Salli\Anaconda3\lib\site-packages\ipykernel_launcher.py:87: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\Salli\Anaconda3\lib\site-packages\ipykernel_launcher.py:88: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\Salli\Anaconda3\lib\site-packages\ipykernel_launcher.py:80: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\Salli\Anaconda3\lib\site-packages\ipykernel_launcher.py:81: ComplexWarning: Casting complex values to real discards the imaginary part


[ 12.00536982+5.37093876e-15j  57.92896322+5.34831479e-15j
 158.26184004+5.34831479e-15j 361.65106602+1.36001933e-08j
 461.25422361+5.34831479e-15j]
lanczos =  265.6464298765655
[  9.89921863+3.27755574e-15j  42.27578221+3.27755574e-15j
 115.99130013-2.37115776e-07j 236.75639996+3.27755574e-15j
 375.00109332+3.27755574e-15j 478.7156082 +3.21235237e-15j]
lanczos =  186.88261660810622
[  8.50484896+5.18296704e-15j  33.38640987+5.21526153e-15j
  89.05390854+5.21526153e-15j 173.00333722+5.21526153e-15j
 323.20235626+5.21526153e-15j 404.60765296+3.68278357e-07j
 466.49810846+5.21526153e-15j]
lanczos =  98.94992814829216
[  7.78236582-1.07410208e-15j  28.83015859-1.07410208e-15j
  76.87632973-1.07410208e-15j 147.67959086-1.06173651e-05j
 235.24571873-1.07410208e-15j 342.59766525-1.07410208e-15j
 419.52646805-1.07410208e-15j 488.69139203-1.07632796e-15j]
lanczos =  72.01125877339143
[  6.71222772+6.21855383e-15j  23.81977895+6.20733115e-15j
  63.16170873+6.20733115e-15j 113.91899831+6.2073311

[  2.01141462+9.48308375e-16j   4.95934475+9.48308375e-16j
   9.42564614+9.48308375e-16j  15.66918498+9.48308375e-16j
  24.52337989+9.48308375e-16j  34.78686402+9.48308375e-16j
  47.06337867-1.96141543e-05j  61.43160263+9.48308375e-16j
  76.42913536+9.48308375e-16j  94.83738062+9.48308375e-16j
 111.2772903 +9.48308375e-16j 129.19148994+9.48308375e-16j
 150.70047951+9.48308375e-16j 167.01956965+9.48308375e-16j
 182.20945651+9.48308375e-16j 197.47416764+9.48308375e-16j
 212.23009104+9.48308375e-16j 225.2737425 +9.48308375e-16j
 250.44243664+9.48308375e-16j 311.27168954+9.48308375e-16j
 361.80973158+9.48308375e-16j 405.60056986+9.48308375e-16j
 454.3948303 +9.48308375e-16j 489.36423293+9.47955051e-16j]
lanczos =  3.2654471061898865e-13


In [ ]:
k = [i*10 for i in range(1,60)]

n = len(k)
mat_array = [0]*n
mat_array_2 = [0]*n
for j in range(n):
    mat_array[j] = (gen_hess(k[j]))
    mat_array_2[j] = gen_rand_mat(np.random.rand(k[j])*500)
time_good = [0]*n
time_bad = [0]*n


In [ ]:
for j in range(len(k)):
    time_good[j] = runtime(QR_decomp_hesse, mat_array[j])
    time_bad[j] = runtime(np.linalg.qr, mat_array_2[j])

In [ ]:
%matplotlib notebook

plt.plot(k, time_good, '-o', label = "good")
plt.plot(k, time_bad, '-o',label = "bad")

plt.legend()
plt.show()

In [ ]:
n_dim_2 = 500 

eigvalues_2 = np.sort(2*np.random.rand(n_dim))
eigvalues_2[-1] = 3
eigvalues_2[-2] = 2.5
eigvalues_2[0] = -1
eigvalues_2[1] = -0.5
L = gen_hermite(eigvalues_2)

j = 10

In [ ]:
lanczos_arr = [[0]]*j

for l in range(j):
    lanczos_arr[l] = lanczos(L,l+2)[1]

In [ ]:
%matplotlib notebook

for l in range(n_dim):
    plt.plot(12,eigvalues[l], 'x')
for l in range(len(lanczos_arr)):
    for k in range(len(lanczos_arr[l])):
        plt.plot(l+2, lanczos_arr[l][k], 'x')
        
plt.axvline(x=11.5)
plt.xlabel("Kyrolvraumdimension, bei 12 Eigenwerte")
plt.ylabel("Wertebereich der Eigenwerte")
plt.show()